# Algorithme Patch Match et parallélisation d'une recherche

Pour le projet d'ELTDM, j'ai décidé de reprendre et d'optimiser un algorithme de complétion d'image. Cette algorithme se décompose en trois étapes, et les trois peuvent être parraléliser. La seconde étape consiste en la recherche dans une image du morceau le plus semblable à un patch trouvé au cours de la première étape. Elle est donc très facilement parallélisable, et c'est sur cette tâche que je me suis concentré pour comprendre les mécanismes mis en jeu au cours de la distribution des calculs. 

Pour présenter le travail réalisé, nous allons tout d'abord regarder l'algorithme dans son ensemble, puis dans un second temps expliquer les différentes approches pour accélérer le calcul, et enfin nous verrons les résultats obtenus, ainsi que les idées de parallélisation des deux autres étapes.

## L'algorithme Patch Match

### 1. La théorie

Tout d'abord regardons de plus près l'algorithme. L'idée de celui-ci vient des deux papiers cités et vise à combler un trou dans une image par un arrière plan réaliste. Prenons l'exemple ci dessous pour mieux comprendre: nous avions à la base une image avec un sauteur à l'élastique. Nous fournissons à l'algorithme cette image en cachant le sauteur, et celui-ci nous renvoit une image où la personne a été remplacée par du ciel. 

<img src="images/results_patch_match_jumper.png" style="width: 1000px;" />

Cet algorithme va remplir le trou en remplissant une petite zone à la fois. Pour ce faire, nous pouvons dicerner trois étapes. 
- Il détermine tout d'abord quelle zone il doit remplir en premier. Pour cela, il s'appuie sur les informations qu'il a déjà, c'est à dire les pixels alentours remplis, mais aussi sur les gradients de couleurs pour prolonger les lignes en premier, avant d'étendre un fond uni. Ainsi, il va remplir tout d'abord les bords du trou, puis se rapprocher de plus en plus du centre.
- Une fois la zone de travail choisi, typiquement une zone comprenant une moitié noire et l'autre moitié de ciel dans notre exemple, l'algorithme va chercher dans le voisinage de cette zone un bout d'image ressemblant. Pour cela, il va calculer pour chaque patch de la même taille que la zone étudiée sa distance à cette zone, et va conserver uniquement le plus proche.
- Enfin, lorsque l'algorithme a à sa disposition la zone à moitié trouée et le patch de référence, il va mixer ces deux patchs pour faire une transition aussi invisible que possible. Pour cela, il va transformer le problème en un graph, dont les noeuds sont les pixels, et dont les liens représentent la difficulté de faire une séparation à cet endroit. Lorsque un lien à un poid fort, l'algorithme va faire en sorte que les noeuds associés proviennent du même patch. Une fois ce graph créé, l'algorithme trouve grâce à un MinCut la séparation la moins couteuse. 

Une fois le patch "mixé" obtenu, l'algorithme le colle à la place de la zone trouée, et recommence ces trois étapes jusqu'à ce que le trou est complétement disparu.

Pour avoir un exemple simple de l'algorithme, vous pouvez lancer le fichier python patchmatch.py se trouvant dans ce même dossier. Pour plus de précision sur l'algorithme, vous pouvez vous référrez à l'autre rapport format PDF, dans lequel la deuxième partie détaille en profondeur l'algorithme.

### 2. Le code

Pour coder cet algorithme, nous avons séparer le code en 5 fichiers distincts, un pour chaque étape (respectivement priority.py, searchmatch.py et mixingpatch.py), un fichier regroupant les fonctions auxiliaires tels que l'affichage d'une image (utils.py) et enfin le fichier principal faisant tourner l'algorithme (patchmatch.py).

Tout d'abord, nous rajoutons une marge sur les bords de l'algorithme pour ne pas avoir de problème d'indice par la suite. Ensuite, nous initialisons une matrice de confiance pour représenter l'information que nous avons, matrice utilisée pendant la première étape et mis à jour à chaque itération. Enfin, nous initialisons le critère sur lequel nous allons boucler.

Ce critère est simple: la présence d'un trou dans l'image repérée grâce à la fonction find_contour de skimage. Dans la boucle, nous choisissons la taille psi des zones considérées, puis nous appelons la fonction get working area du fichier priority. Celle-ci va calculer pour chaque point du contour un score de priorité, et retourner le point sur lequel ce score est le plus grand.

Ensuite, nous appelons la fonction findbestmatch, qui se charge de trouver la zone la plus ressemblante au patch trouvé. Dans le cas non parallélisé, nous bouclons simplement sur tous les pixels dans un voisinage de taille fixe autour du point trouvé, nous calculons à chaque fois la distance entre les deux zones considérées, et nous stockons la plus proche. Ce code est la première partie du fichier searchmatch.py. La suite est la parallélisation du calcul.

Enfin, nous appelons get_mix_patch du fichier mixingpatch qui va créer le graph, appliquer le MinCut, se charger de la gestion des anciennes coutures, flouter la couture créée, et retourner la patch final.

Enfin, il ne nous reste plus qu'à coller le patch, mettre à jour la matrice de confiance, le critère sur lequel on boucle et le contour du trou si trou il y a.

## Parallélisation de la recherche

L'algorithme tel que présenté si dessus marche très bien. Cependant, il est très long à tourner. Pour donner un ordre de grandeur, nous avons du attendre une dizaine d'heure pour obtenir le résultat ci dessus. Cette lenteur vient principalement des étapes deux et trois. L'étape de recherche est facilement parallélisable. Nous nous y sommes dons intéressés en priorité !

La première chose à noter est que nous travaillons en multi-processes et non en multi-thread, pour éviter les problèmes avec le GIL.

Pour paralléliser le calcul, nous pouvons penser à trois approches: en ayant recours à des Queues, en partageant l'image à la main entre les différents coeurs ou en mixant ces deux approches. 

### 1. Les Queues

La première approche se trouve dans le bas du fichier searchmatch.py. Elle consiste à stocker dans une queue tous les centres des zones à tester. Une fois cette queue initialisée, les différents worker viennent piocher un élément dans cette queue et calculer la distance par rapport à la zone de référence. Chaque worker stocke dans son espace mémoire l'élément qu'il a vu le plus ressemblant. Une fois la queue vide, chaque worker dépose l'élément qu'il a trouvé dans une seconde Queue. Il nous suffit alors de prendre l'élément le plus proche dans cette seconde Queue. Cette démarche est illustrée dans le schéma suivant.

<img src="images/Approche1.svg" style="width: 400px;" />

Cette technique propose le gros avantage d'exploiter tous les workers de manière équivalente. En effet, le traitement n'a pas toujours le même couts selon l'endroit de l'image que l'on examine. Néanmoins, la file d'attente permet à chaque worker de prendre un nouvel élément à traiter à chaque fois qu'il est désoeuvré. 

D'un autre coté, cette technique à l'inconvénient de copier l'image plusieurs fois. En effet, comme les processes ont chacun leur propre espace mémoire, l'image initiale doit être répliquer autant de fois qu'il y a de workers. Nous travaillons ici avec des images de taille moyenne, donc se problème n'est pas important dans notre cas. Cependant, si nous voulons étendre cet algorithme à des vidéos, la gestion de la mémoire deviendra primordial.

### 2. Se passer des Queues

Un autre inconvénient de l'approche précédente est le fait que tous les workers soient reliés à la même Queue en entrée. En effet, comme nous devons faire le calcul de la distance pour un grand nombre de zones (entre 140 000 pour les images simples et 650 000 pour les images testées les plus complexes), si nous avons un PC avec 4 coeurs, chaque worker viendra prendre entre 35 000 et 175 000 éléments dans cette Queue. Ainsi, nous pouvons penser que nous perdons beaucoup de temps pour aller récupérer les éléments un par un. 

Pour combler se problème, nous pouvons éviter de créer une Queue en divisant l'image en W zones si nous avons W workers, et en attribuant à chaque worker une unique zone. Les différents processes devront alors uniquement communiquer pour donner leur résultat une fois leur travail fini. Cette aproche est illustrée dans le schéma suivant:

<img src="images/Approche2.svg" style="width: 1000px;" />

Cette méthode se trouve dans le fichier withoutqueuesearchmatch.py.


Cette technique présente donc le gros avantage de réduire les couts de communication au minimum. Néanmoins, cela s'accompagne d'un gros défaut: les workers ne sont pas tous utilisé uniformément. En effet, lors de la recherche de la zone la plus ressemblante au patch initial, si il y a un trou dans la zone considérée, l'algorithme passe à la zone suivante sans calculer de distance. Ainsi, le temps de calcul varie beaucoup pour une seule opération, ce qui peut entraîner qu'un worker finisse de calculer bien avant les autres.

Un autre avantage de cette technique est que l'image initiale n'est pas répliquée pour chaque worker. En effet, chaque worker a uniquement besoin d'une petite partie de l'image, et seules les bords des zones sont doublées entre les workers.

### 3. Mixer les deux approches

Nous avons donc vu deux différentes approches nous permettant de paralléliser la recherche de la zone la plus ressemblante. Comme les défauts de l'une sont les avantages de l'autre, et vice et versa, nous pouvosn penser que nous obtiendrons de meilleurs résultats en mélangeant les deux méthodes. 

Pour ce faire, nous allons encore une fois diviser l'image initiale en plusieurs parties, mais cette fois nous prendrons plus de parties que de workers. Nous mettrons chaque partie découpée dans une queue, et les workers viendront piocher dans cette queue pour effectuer les calculs nécessaires. Comme dans les méthodes précédentes, chaque worker stockera en local le meilleur résultat qu'il a vu, et l'enverra une fois son travail fini dans une queue de sortie. Il ne nous restera qu'à prendre le meilleur résultat ! Encore une fois, nous pouvons illutrer cette approche par un schéma. 


<img src="images/Approche3.svg" style="width: 1000px;" />

Cette méthode se trouve dans le fichier mixsearchmatch.py.

Nous pouvons penser que grâce à cette approche, les coûts de communication pour prendre un élément de la queue d'entrée seront moins apportant que dans la première approche, mais aussi que le grand nombre de partie découpée permettra de diminuer l'écart de temps de calcul entre les différents calculs.

Cependant, il peut y avoir un défaut caché derrière cette méthode selon le nombre de découpages. Les zones d'overlap entre les différents worker peuvent entrainer des couts mémoire très important. En effet, imaginons que nous étudions des patchs de taille 51, que l'image est beaucoup plus grande à la base, et que nous la découpons selon chaque ligne. Dans ces conditions, chaque ligne sera copiée 51 fois, puisqu'elle doit être présente dans les 25 zones du dessus, dans les 25 zones en dessous, et dans la sienne. Ainsi, nous devrons stocker non plus une image initiale, mais l'équivalent de 51 iamges. Encore une fois, dans le cadre d'image de taille moyenne, cela ne constitue pas un problème majeur, mais si nous augmentons la taille des patchs et que nous travaillons sur des vidéos, cela deviendra rapidement critique !

## Résultats et suite

### 1. Résultats

Nous avons donc vu trois approches, et nous sommes en droit de penser que chacune de ces méthodes donnera le résultat attendu plus rapidement que dans le cas non parallélisé. Pour tester ces algorithmes, nous allons comparer le temps que ces différents codes mettent pour trouver la partie la plus ressemblante à un patch de taille 59x59 dans une zone de 200x200. Cela signifie qu'il y a 40 000 zones possibles pour lesquelles il faut calculer une distance.

Tout d'abord, comparons ces différentes approches:

|             Méthode            | Basique | Approche 1 | Approche 2 | Approche 3 |
|:------------------------------:|:-------:|:----------:|:----------:|:----------:|
| Temps d'execution  en secondes |  157.14 |    69.53   |    54.18   |    52.12   |

A partir de ce tableau, nous pouvons émettre plusieurs remarques. Tout d'abord, la parallélisation de ce calcul est efficace puisque nous réduisons significativement le temps de calcul, peut import la méthode utilisée. Néanmoins, nous pouvons remarquer que nous avons utilisé 4 coeurs, mais nous avons divisé le temps de calcul par 3 uniquement. La méthode n'est donc pas encore optimale !

Nous pouvons aussi remarquer que, comme nous pouvions nous y attendre, la troisième approche est la meilleure. Cependant, le gain est faible par rapport à la seconde méthode. Nous pouvons expliquer cela par le fait que le trou fait dans l'image était plutôt petit, donc que chaque worker a eu à peu de chose près le même travail à faire. Il faudrait essayer de comparer ces méthodes sur un cas réel. 

Dernière remarque, nous pouvons remarquer que les approches 2 et 3 sont nettement meilleures que la première. Cela confirme notre soupçon de "bouchon" à la sortie de la queue, bouchon entrainant une perte d'efficacité.

Un autre axe d'étude intéressant est de voir l'impact du nombre de zone dans la troisième approche. Les résultats d'une telle expérimentation sont présentés ci-dessous:

| Nombre de zones | Temps d'éxécution en secondes |
|:---------------:|:-----------------------------:|
|        4        |             58.97             |
|        5        |             54.36             |
|        6        |             52.24             |
|        7        |             72.22             |
|        8        |             56.51             |
|        9        |             56.99             |
|        10       |             56.18             |
|        11       |             55.48             |
|        12       |             60.46             |
|        13       |             61.30             |
|        14       |             59.05             |
|        15       |             56.05             |
|        16       |             58.84             |
|        17       |             59.86             |
|        18       |             59.80             |
|        19       |             60.67             |
|        20       |             60.95             |

Nous pouvons voir qu'en dessous de 11 zones, les temps d'éxécution sont semblables. De plus, les résultats semblent cohérent avec ceux de la méthode 2. En effet, pour 4 zones, cette troisième approche est plus lente, ce qui est logique puisque qu'elle requiert la création d'une queue en plus.

Nous pouvons aussi remarquer que, au dessus de 16 zones, le temps de récupération des éléments dans la Queue revient et augmente le temps d'éxécution de la recherche.

Nous trouvons des résultats similaires lorsque nous diminuons le nombre de coeurs utilisés, mis à part le point aberrant pour 7 zones. L'ordinateur a du faire un autre calcul en même temps.

Pour dernière remarque sur ces résultats, nous pouvons ajouter qu'ils doivent être pris avec précautions. En effet, en lancant deux fois le même calcul, les résultats diffèrent d'un écart pouvant froler la dizaine de secondes.

### 2. Prolongations ?

Avec ce projet, nous avons étudié le cas particulier de l'étape 2 de notre algorithme, lorsque l'on recherche une zone ressemblante dans une grande image. Cependant, l'algorithme reste long à tourner à cause de la troisième étape. Celle-ci étant un algorithme de MinCut, elle se résume à trouver les valeurs singulières d'une matrice. Après quelque recherche rapide sur le net, il s'avère que ce calcul peut être parallélisé lui aussi. Les méthodes employées sont plus complexes que celles vues ici, et par conséquent moins visuels. Cependant, cette parallélisation est sans doute la prochaine voie d'amélioration de notre algorithme.

La première étape peut être elle aussi distribuée, puisque le même calcul est fait sur chaque point du contour. Cela n'est pas très compliqué, et selon la date/l'heure où vous lirez ces lignes, cette fonctionnalité sera peut être déjà codée !

Si nous arrivons à rendre le calcul de chacune de ces étapes efficace, la prochaine étape sera de s'attaquer à des vidéos, choses qui ne pose pas de très gros problèmes théoriquement avec cet algorithme, mais qui rajoute énormément en complexité, puisque nous devrons travailler avec des patchs en 3D, chercher dans une large zone 3D, créer un graph 3D ... En un mot, il reste encore du travail !

## Conclusion

Avec ce projet, nous avons donc pu nous familiariser avec la parallélisation "manuelle" des calculs, en utilisant des packages tels que threading et multiprocessing. En faisant nos premiers pas avec ces packages, nous sommes tombés dans les pièges classiques, notamment avec le GIL. C'est pourquoi nous avons fait le choix de travailler avec des processes.

Une fois ce choix fait, nous avons pu expérimenter plusieurs manière d'approcher la parallélisation, voir les défauts des unes et des autres et ainsi comparer les résultats sur un cas simple.

Au final, nous avons bien optimiser un tier de l'algorithme global. Reste maintenant à coder le reste !

## Sources
- Criminisi, A., Pérez, P., & Toyama, K. (2004). Region filling and object removal by exemplar-based image inpainting. IEEE Transactions on image processing, 13(9), 1200-1212.

- Kwatra, V., Schödl, A., Essa, I., Turk, G., & Bobick, A. (2003, July). Graphcut textures: image and video synthesis using graph cuts. In ACM Transactions on Graphics (ToG) (Vol. 22, No. 3, pp. 277-286). ACM.